In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
data_dir = (Path().cwd().parents[0] / 'data'/ 'pixelgen').absolute() / 'Tcell' / 'graphs'


In [9]:
import PPIGraph

In [10]:
condition_mapping = {'Control': 0, 'Stimulated': 1}

# Read per cell data

In [11]:
from torch_geometric.loader import DataLoader

In [12]:
# Dataset
dataset = PPIGraph.GraphDatasetPixelgen(data_dir, 'raw', 'pt',  condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)



In [13]:
dataset

GraphDatasetPixelgen(932)

In [14]:
dataset[0]

Data(edge_index=[2, 7188], pos=[1200, 3], labels=[1200, 80], weight=[7188], condition=0, id=0, train_mask=[1200], test_mask=[1200], x=[1200, 80], y=[1], edge_weight=[7188], name='0_0.gpt')

# ML model

In [15]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [16]:
condition = 'ML'
project_name = f'Pixelgen_{condition}'

In [17]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [18]:
import torch 

data = torch.empty((0, 80))
label = []

for graph in dataset:
    new_row =graph.x.sum(axis = 0).unsqueeze(0)
    data = torch.cat((data, new_row), dim=0)
    label.append(graph.condition)


In [22]:
df = pd.DataFrame(data)
df['Condition'] = label

In [18]:
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import HistGradientBoostingClassifier


In [19]:
scaler = StandardScaler()

# Run model on cell count
X = data.numpy()
X = scaler.fit_transform(X)
y = np.array(label)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.97861
auc,0.97776
b_accuracy,0.97776
f1,0.97674


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.98396
auc,0.98611
b_accuracy,0.98611
f1,0.98137


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.96774
auc,0.96534
b_accuracy,0.96534
f1,0.96341


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.96774
auc,0.96736
b_accuracy,0.96736
f1,0.96907


...

C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.96774
auc,0.96849
b_accuracy,0.96849
f1,0.96471


C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.97849
auc,0.97882
b_accuracy,0.97882
f1,0.97895


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.97849
auc,0.97724
b_accuracy,0.97724
f1,0.9759
